In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from itertools import permutations
from collections import defaultdict
import random

In [ ]:
def sig_func(x,y):
    return(1/(1+np.exp(-(np.dot(x,y)))))

In [ ]:
global board_size
board_size = 100

In [ ]:
# Funtion to build the initial cell list.  NN is built around sight/input of 5, movement/output of 3.
def cell_list():
    start_coords = np.random.choice(range(3,46),board_size)
    cells  = defaultdict(dict)
    
    for i in range(0,board_size,2):
        name = str(random.getrandbits(128))
        cells[name] = {}
        
        coords = [start_coords[i],start_coords[i+1]]
        disc_weights = 2*np.random.random((5*5,3*3))-1
        
        cells[name]['size'] = 3
        cells[name]['coords'] = coords
        cells[name]['disc_weights'] = disc_weights

    return(cells)

In [ ]:
# Create the cell list.
cells = cell_list()

In [ ]:
# Create the board.
X = np.zeros(shape=(board_size,board_size))

In [ ]:
# Get the coodinates for the input.
L = [0, 0, 1, 1, -1, -1, 2, 2, -2, -2]
keep = [0,0]
for comb in permutations(L, r=2):
    keep = np.vstack((keep, comb))

In [ ]:
# Setup plotting.
fig = plt.figure(figsize=(10, 10))
ax = fig.add_axes([0, 0, 1, 1], xticks=[], yticks=[], frameon=False)
fig.show()
fig.canvas.draw()    

# Run indefinately.
while(True):
    
    # Plotting.
    ax.clear()
    
    # Take turns through the cell list.
    for i in list(cells):

        # Input.
        test_in = []
        for perm in np.mod(cells[i]['coords'] + np.unique(keep,axis=0), board_size):
            test_in = np.append(test_in,X[perm[0],perm[1]])
            
        # Output.
        test_out = sig_func(test_in,cells[i]['disc_weights'])
        d = test_out.argmin()

        # Enviroment logic:
        
        # Stay and Grow.
        if(d==4):               
            if(cells[i]['size'] < 6):
                cells[i]['size'] += 1
                print("{0} grew to size {1}.".format(i, cells[i]['size']))
                
        # Move.
        else:
            if(d==0):
                new_coord = [-1,1]
            if(d==1):
                new_coord = [0,1]
            if(d==2):
                new_coord = [1,1]
            if(d==3):
                new_coord = [-1,0]

            if(d==5):
                new_coord = [1,0]
            if(d==6):
                new_coord = [-1,-1]
            if(d==7):
                new_coord = [0,-1]
            if(d==8):
                new_coord = [1,-1]
            
            # Split.
            if(cells[i]['size'] > 5):
                
                # Half its size.
                cells[i]['size'] = 3
                
                # Give the baby attributes and mutate the NN.
                baby_name = str(random.getrandbits(128))
                cells[baby_name]['size'] = cells[i]['size']
                cells[baby_name]['coords'] = cells[i]['coords']
                cells[baby_name]['disc_weights'] = cells[i]['disc_weights']
                cells[baby_name]['disc_weights'][np.random.choice(5*5),np.random.choice(3*3)] == 2*np.random.random(1)-1
                
                print("{0} underwent mitosis.  Gave birth to {1}.".format(i, baby_name))
            
            # If it moved.
            if(X[np.mod(np.add(cells[i]['coords'] , new_coord), board_size)[0],np.mod(np.add(cells[i]['coords'] , new_coord), board_size)[1]] == 0):
                
                # Clear where it was.
                X[cells[i]['coords'][0],cells[i]['coords'][1]] = 0
                
                # Give new coordinate and fill it in.
                cells[i]['coords'] = np.mod(np.add(cells[i]['coords'] , new_coord), board_size)
                X[cells[i]['coords'][0],cells[i]['coords'][1]] = abs(sum(sum(cells[i]['disc_weights'])))
                
            else:
                # It gets bumped.
                cells[i]['size'] -= 1
                print("{0} was injured to size {1}!".format(i, cells[i]['size']))
                
                # Handle death.
                if(cells[i]['size'] < 1):
                    try:
                        # Clear where it was and delete from list.
                        X[cells[i]['coords'][0],cells[i]['coords'][1]] = 0
                        del cells[i]
                        print("{0} died!.".format(i))
                    except KeyError:
                        pass
                    
    # Draw heatmap.
    ax.matshow(X, cmap = 'magma')
    fig.canvas.draw()